In [1]:
from selenium import webdriver
import re
import pandas as pd
import time
import os

In [2]:
indicators = {
    'NY.GDP.MKTP.CD': 'GDP_current_USD',
    'NY.GDP.PCAP.CD': 'GDP_per_capita',
    'FP.CPI.TOTL.ZG': 'Inflation',
    'SL.UEM.TOTL.ZS': 'Unemployment',
    'SP.POP.TOTL': 'Population',
    'NE.EXP.GNFS.ZS': 'Exports_pct_GDP'
}

countries = {
    'Afghanistan': 'AF',
    'Bangladesh': 'BD',
    'Bhutan': 'BT',
    'India': 'IN',
    'Maldives': 'MV',
    'Nepal': 'NP',
    'Pakistan': 'PK',
    'Sri Lanka': 'LK'
}

# Years to scrape
years_to_scrape = [2018, 2019]

In [3]:
def scrape_multiple_years(indicator_code, country_code, country_name, indicator_name, years):
    """
    Scrape data for multiple years from a single page
    Returns: list of dicts with data
    """
    
    print(f"  {country_name:12} - {indicator_name:20}...", end=" ", flush=True)
    
    driver = None
    
    try:
        driver = webdriver.Safari()
        
        url = f"https://data.worldbank.org/indicator/{indicator_code}?locations={country_code}"
        driver.get(url)
        time.sleep(6)
        
        html = driver.page_source
        
        driver.quit()
        driver = None
        
        results = []
        
        for year in years:
            # Pattern: "YEAR",["^ ","^2","atom","^3",VALUE,"^21",true]
            pattern = f'"{year}",\\["[^"]*","[^"]*","[^"]*","[^"]*",([0-9.eE+\\-]+)'
            
            matches = re.findall(pattern, html)
            
            if matches:
                value = float(matches[0])
                results.append({
                    'country_name': country_name,
                    'country_code': country_code,
                    'indicator_code': indicator_code,
                    'indicator_name': indicator_name,
                    'year': year,
                    'value': value
                })
        
        # Print results
        if results:
            year_values = [f"{r['year']}:✅" for r in results]
            print(f"{', '.join(year_values)}")
        else:
            print("⚠️ No data")
        
        return results
            
    except Exception as e:
        print(f"❌ Error: {str(e)[:40]}")
        return []
    
    finally:
        if driver is not None:
            try:
                driver.quit()
            except:
                pass
        time.sleep(1)

print("✅ Scraping function defined")

✅ Scraping function defined


# Main Scraping

In [4]:
print("="*80)
print("WORLD BANK 2018-2019 DATA SCRAPER")
print("="*80)
print(f"\n⏱️  Estimated time: ~5 minutes")
print(f"📦 Total pages to visit: {len(indicators) * len(countries)}\n")

all_data = []
total = len(indicators) * len(countries)
current = 0

for indicator_code, indicator_name in indicators.items():
    
    print(f"\n{'─'*80}")
    print(f"📈 {indicator_name}")
    print('─'*80)
    
    for country_name, country_code in countries.items():
        
        current += 1
        print(f"[{current}/{total}] ", end="")
        
        data = scrape_multiple_years(
            indicator_code,
            country_code,
            country_name,
            indicator_name,
            years_to_scrape
        )
        
        if data:
            all_data.extend(data)
        
        time.sleep(2)
    
    time.sleep(3)

print("\n" + "="*80)
print("🎉 SCRAPING COMPLETE!")
print("="*80)

WORLD BANK 2018-2019 DATA SCRAPER

⏱️  Estimated time: ~5 minutes
📦 Total pages to visit: 48


────────────────────────────────────────────────────────────────────────────────
📈 GDP_current_USD
────────────────────────────────────────────────────────────────────────────────
[1/48]   Afghanistan  - GDP_current_USD     ... 2018:✅, 2019:✅
[2/48]   Bangladesh   - GDP_current_USD     ... 2018:✅, 2019:✅
[3/48]   Bhutan       - GDP_current_USD     ... 2018:✅, 2019:✅
[4/48]   India        - GDP_current_USD     ... 2018:✅, 2019:✅
[5/48]   Maldives     - GDP_current_USD     ... 2018:✅, 2019:✅
[6/48]   Nepal        - GDP_current_USD     ... 2018:✅, 2019:✅
[7/48]   Pakistan     - GDP_current_USD     ... 2018:✅, 2019:✅
[8/48]   Sri Lanka    - GDP_current_USD     ... 2018:✅, 2019:✅

────────────────────────────────────────────────────────────────────────────────
📈 GDP_per_capita
────────────────────────────────────────────────────────────────────────────────
[9/48]   Afghanistan  - GDP_per_capita   

In [5]:
df = pd.DataFrame(all_data)
df.head()

,country_name,country_code,indicator_code,indicator_name,year,value
0,Afghanistan,AF,NY.GDP.MKTP.CD,GDP_current_USD,2018,1.805322e+10
1,Afghanistan,AF,NY.GDP.MKTP.CD,GDP_current_USD,2019,1.879944e+10
2,Bangladesh,BD,NY.GDP.MKTP.CD,GDP_current_USD,2018,3.213628e+11
3,Bangladesh,BD,NY.GDP.MKTP.CD,GDP_current_USD,2019,3.512317e+11
4,Bhutan,BT,NY.GDP.MKTP.CD,GDP_current_USD,2018,2.583336e+09


In [6]:
df.shape

(94, 6)

In [7]:
df.describe()

,year,value
count,94.000000,9.400000e+01
mean,2018.500000,7.655259e+10
std,0.502681,4.052337e+11
min,2018.000000,-1.333734e-01
25%,2018.000000,5.572619e+00
50%,2018.500000,4.939699e+02
75%,2019.000000,3.466080e+07
max,2019.000000,2.835606e+12


In [8]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   country_name    94 non-null     str    
 1   country_code    94 non-null     str    
 2   indicator_code  94 non-null     str    
 3   indicator_name  94 non-null     str    
 4   year            94 non-null     int64  
 5   value           94 non-null     float64
dtypes: float64(1), int64(1), str(4)
memory usage: 4.5 KB


In [9]:
df.isna().sum()

country_name      0
country_code      0
indicator_code    0
indicator_name    0
year              0
value             0
dtype: int64

In [10]:
df.dropna()

,country_name,country_code,indicator_code,indicator_name,year,value
0,Afghanistan,AF,NY.GDP.MKTP.CD,GDP_current_USD,2018,1.805322e+10
1,Afghanistan,AF,NY.GDP.MKTP.CD,GDP_current_USD,2019,1.879944e+10
2,Bangladesh,BD,NY.GDP.MKTP.CD,GDP_current_USD,2018,3.213628e+11
3,Bangladesh,BD,NY.GDP.MKTP.CD,GDP_current_USD,2019,3.512317e+11
4,Bhutan,BT,NY.GDP.MKTP.CD,GDP_current_USD,2018,2.583336e+09
...,...,...,...,...,...,...
89,Nepal,NP,NE.EXP.GNFS.ZS,Exports_pct_GDP,2019,7.779936e+00
90,Pakistan,PK,NE.EXP.GNFS.ZS,Exports_pct_GDP,2018,8.581800e+00
91,Pakistan,PK,NE.EXP.GNFS.ZS,Exports_pct_GDP,2019,9.390864e+00
92,Sri Lanka,LK,NE.EXP.GNFS.ZS,Exports_pct_GDP,2018,2.144625e+01


In [11]:
df.duplicated().sum()

np.int64(0)

In [12]:
df.drop_duplicates()

,country_name,country_code,indicator_code,indicator_name,year,value
0,Afghanistan,AF,NY.GDP.MKTP.CD,GDP_current_USD,2018,1.805322e+10
1,Afghanistan,AF,NY.GDP.MKTP.CD,GDP_current_USD,2019,1.879944e+10
2,Bangladesh,BD,NY.GDP.MKTP.CD,GDP_current_USD,2018,3.213628e+11
3,Bangladesh,BD,NY.GDP.MKTP.CD,GDP_current_USD,2019,3.512317e+11
4,Bhutan,BT,NY.GDP.MKTP.CD,GDP_current_USD,2018,2.583336e+09
...,...,...,...,...,...,...
89,Nepal,NP,NE.EXP.GNFS.ZS,Exports_pct_GDP,2019,7.779936e+00
90,Pakistan,PK,NE.EXP.GNFS.ZS,Exports_pct_GDP,2018,8.581800e+00
91,Pakistan,PK,NE.EXP.GNFS.ZS,Exports_pct_GDP,2019,9.390864e+00
92,Sri Lanka,LK,NE.EXP.GNFS.ZS,Exports_pct_GDP,2018,2.144625e+01


In [14]:
df.to_csv('../data/processed/south_asia_economic_metric_2018_to_2019.csv')
print('Clean WDI South Asia')

Clean WDI South Asia
